In [42]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences
from time import sleep
from tqdm import tqdm  # optional: pip install tqdm


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")

SYSTEM_PROMPT_IMAGINE = """
Your task is to take a human imaginative directive and translate it into a coherent, structured list of domains, technologies, and disciplines that could be explored to realize or approximate that imaginative directive.
The goal is to break down the directive into actionable or researchable avenues across science, technology, psychology, education, or other relevant fields.
Each entry should be self-contained, meaning it should be understandable on its own, and clearly tied back to the core imaginative directive.
"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}



def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt

def summarize(excerpt):
    usr_msg = f"Summarize the following excerpt.\nExcerpt:\n{excerpt}"
    summary = ask("", usr_msg)
    print(summary)


load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "o4-mini-2025-04-16")

DB_FILE = "library.sqlite"
MAX_RESULTS = 100

In [2]:
KEYWORDS = ["hindgut morphogenesis"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
hindgut = {}
for r in search_results:
    title, text = r['title'], r['text']
    hindgut[title] = text
    print(title, len(text))

Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila 86702
It Takes Guts: The Drosophila Hindgut as a Model System for Organogenesis 99729


In [7]:
KEYWORDS = ["hindgut"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# hindgut = {}
for r in search_results:
    title, text = r['title'], r['text']
    hindgut[title] = text
    print(title, len(text))

Electrical Stimulation for Pelvic Floor Disorders 734489
None 5149195
Fundamental Neuroscience (3rd edition) 5149195
Clinical Emergency Medicine 1448602
Harrison's Principles of Internal Medicine 15th Edition 16901303
The Drosophila actin nucleator DAAM is essential for left-right asymmetry 70296
Myosin1D is an evolutionarily conserved regulator of animal left–right asymmetry 66732
An unconventional myosin in Drosophila reverses the default handedness in visceral organs 28626
Chiral cell sliding drives left-right asymmetric organ twisting 82971
Cell Chirality Drives Left-Right Asymmetric Morphogenesis 43287
Chirality in Planar Cell Shape Contributes to Left-Right Asymmetric Epithelial Morphogenesis 18105
Vertebrate myosin 1d regulates left–right organizer morphogenesis and laterality 54138
Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila 86702
Genital disc growth in Drosophila 53597
Understanding Laterality 

In [8]:
KEYWORDS = ["active tension network"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# hindgut = {}
for r in search_results:
    title, text = r['title'], r['text']
    hindgut[title] = text
    print(title, len(text))

Active Tension Network Model in Epithelial Tissues 32334
The Geometric Basis of Epithelial Convergent Extension 188479
Generating Active T1 Transitions through Mechanochemical Feedback 117183


In [9]:
rem = ["Electrical Stimulation for Pelvic Floor Disorders", None, "Fundamental Neuroscience (3rd edition)", "Clinical Emergency Medicine", "Harrison's Principles of Internal Medicine 15th Edition"]
for r in rem:
    if r in hindgut:
        del hindgut[r]

for title, text in hindgut.items():
    print(title)

Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila
It Takes Guts: The Drosophila Hindgut as a Model System for Organogenesis
The Drosophila actin nucleator DAAM is essential for left-right asymmetry
Myosin1D is an evolutionarily conserved regulator of animal left–right asymmetry
An unconventional myosin in Drosophila reverses the default handedness in visceral organs
Chiral cell sliding drives left-right asymmetric organ twisting
Cell Chirality Drives Left-Right Asymmetric Morphogenesis
Chirality in Planar Cell Shape Contributes to Left-Right Asymmetric Epithelial Morphogenesis
Vertebrate myosin 1d regulates left–right organizer morphogenesis and laterality
Genital disc growth in Drosophila
Understanding Laterality Disorders and the Left-Right Organizer: Insights from Zebrafish
Generating Active T1 Transitions through Mechanochemical Feedback
Left–right Myosin-Is, Myosin1C, and Myosin1D exhibit distinct single

In [11]:
SYSTEM_PROMPT_LINK = """
You are a “cross-domain cartographer.”
Your job is to discover and describe conceptual and practical bridges between two input texts.
• Assume at least one text concerns Drosophila hindgut development (genes, axes, left-right asymmetry, etc.).
• Treat the other text as a source of additional ideas—it might discuss cognition, physics, economics, art, or any other field.
• Your task is to show how insights, models, or metaphors from the second text can feed or enrich the first, with special attention to hindgut morphogenesis if it is present.
• Deliver your answer in four clearly labelled sections:
Conceptual Parallels – key analogies or shared principles (3–5 bullets).
Mechanistic Bridges – concrete pathways for translating methods, models, or data between domains (3–5 bullets).
Experimental or Modeling Suggestions – actionable ideas to test or implement the bridges (2–4 bullets).
Story/Metaphor Hook – a brief narrative or image (<150 words) that captures the link for a broad audience.
• Write in concise, information-dense prose; use specific gene names, pathways, or theoretical terms where appropriate.
• Avoid unnecessary fluff; favour clarity and scientific rigor.
"""

USER_PROMPT_LINK = """
Input Text A (primary):
{primary_text}

Input Text B (secondary):
{secondary_text}

Task:
Find ways that the information in Text B can feed Text A.
Specifically, what strategies can link the ideas from Text B to the hindgut morphogenesis themes in Text A (or vice-versa if only Text B covers hindgut)?
Please provide:
1. Conceptual parallels
2. Mechanistic bridges
3. Experimental or modelling suggestions
4. A short story/metaphor hook
"""

In [12]:
n = random.randint(0, len(hindgut)-1)
title1, text1 = list(hindgut.keys())[n], list(hindgut.values())[n]
excerpt1 = sample_text(text1, 10000)
print(title1)

m = random.randint(0, len(hindgut)-1)
title2, text2 = list(hindgut.keys())[m], list(hindgut.values())[m]
excerpt2 = sample_text(text2, 10000)
print(title2)

Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila
Active Tension Network Model in Epithelial Tissues


In [14]:
summarize(excerpt1)
print("\n\n")
summarize(excerpt2)

The excerpt discusses research on Drosophila hindgut morphogenesis, focusing on two distinct cellular and junctional dynamics—cell sliding and cell intercalation. The excerpt highlights several experimental findings: E-cad mutant and Myo1D-overexpressing embryos exhibit different cell dynamics, with E-cad mutants showing no significant change in cell intercalation but altered cell boundary rotation. Conversely, embryos overexpressing Myo1D show an enhanced bias in cell boundary rotation. It suggests that cell sliding and intercalation are independent processes. Par-3/baz mutants exhibit defects in hindgut elongation due to reduced intercalation, while sliding and rotation remain unaffected. These findings imply that different genetic and cellular influences distinctly regulate the two mechanisms, underlying their concurrent but independent execution. Additionally, a computational model suggests these dynamics are supported by contractile forces with different polarities, indicating a c

In [16]:
sys_msg = SYSTEM_PROMPT_LINK
usr_msg = USER_PROMPT_LINK.format(primary_text=excerpt1, secondary_text=excerpt2)
resp = ask(sys_msg, usr_msg)
print(resp)

1. **Conceptual Parallels:**
   - **Dynamic Equilibrium:** Both texts discuss mechanisms where dynamic, transient processes contribute to larger structural changes (e.g., medial myosin pulses vs. hindgut rotation).
   - **Cellular Tension and Morphogenesis:** Text B's tension net hypothesis parallels the role of E-cadherin and Myo1D in Text A, both central to managing tensions during morphological transformations.
   - **Independent Processes in Morphogenesis:** Similar to the independent regulation of convergent extension and cell sliding in Text A, the ATN model in Text B suggests independent control mechanisms for tension and myosin dynamics.
   - **Chirality and Rotation:** The idea of cell chirality causing rotational movement in Text A aligns with anisotropic deformations from balanced tension nets in Text B.
   - **Geometry and Functionality:** Both texts utilize geometry—angle measurements in Text A and isogonal deformation in Text B—to explain cellular behavior influencing tis

In [47]:
sNARRATIVE = """
You are a “hindgut morphogenesis story-smith.”  
Your job is to transform one technical excerpt on Drosophila hindgut development into a single mechanism-driven narratives on one specific aspect of hindgut development.  
For this narrative, output **three labelled sub-sections**:

1. **Core Plot** – ≤120 words, written like a scene: who (genes/proteins), what (cell moves), why (developmental purpose).  
2. **Mechanistic Highlights** – 3-5 bullet points naming specific genes, motors, junctional complexes, forces, or geometrical cues that power the plot.  
3. **Open Questions / Test Ideas** – 1-2 concrete experiments, perturbations, or modelling approaches that would probe or challenge the story.

Generate the a narrative on the following :

Rotation dynamics 

Special instructions:
- Write with scientific precision yet use active verbs and imagery; aim for the style of a concise research vignette, not fluff.  
- If a requested detail is missing from the excerpt, infer cautiously from standard literature (flag assumptions with “(inferred)”).

"""

uNARRATIVE = """
Input text on hindgut morphogenesis:
{excerpt}
Task:
Using the system instructions, craft the requested narrative—Rotation dynamics—each with Core Plot, Mechanistic Highlights, and Open Questions/Test Ideas.
"""




In [43]:
sys_msg = SP_STORY
usr_msg = UP_STORY.format(excerpt=excerpt1)
resp = ask(sys_msg, usr_msg)
print(resp)

1. Core Plot  
In the living hindgut epithelium, E-cadherin tethers neighboring cells as Myosin1D motors pulse along jagged junctions. Over 60 minutes, individual cell boundaries swivel around the tube’s long axis—dextral chirality emerges as red-stained edges rotate counterclockwise, propelling the entire gut into a rightward coil. When E-cad is lost, boundaries slip without torque and rotation bias vanishes; when Myo1D is overabundant, junctions spin more vigorously, exaggerating the twist. This coordinated rotation sculpts left–right asymmetry, converting microscopic junctional torques into the macroscopic dextral hook of the hindgut.

2. Mechanistic Highlights  
• E-cadherin at adherens junctions sets frictional resistance, anchoring edges to transmit chiral torque.  
• Myosin1D enrichment on rightward-tilted T3–T1 junctions drives anisotropic contractile pulses (inferred from overexpression data).  
• Junctional remodeling (T3→T1 transitions) proceeds independently of rotation, pr

In [22]:
SP_SHARPEN = """
You are a “scientific narrative editor.”  
Your job is to tighten, clarify, and deepen a set of mini-narratives about Drosophila hindgut morphogenesis.

For each mini-narrative you receive, do **all** of the following:

1. **Score** it (0-10) on five axes:
   • Fidelity to brief • Scientific precision • Mechanistic depth • Experimental imagination • Narrative punch  
   Present scores in a one-line table.

2. **Rewrite** the piece in ≤120 words, keeping the original structure (Core Plot / Mechanistic Highlights / Open Questions), but
   • Replace vague phrases with quantitative or anatomical specifics wherever possible.  
   • Introduce at least one concrete observable (e.g., rotation rate, chirality index).  
   • Remove filler words; use vivid but accurate verbs.  
   • Preserve every key idea; do not invent new biology unless clearly flagged “(inferred)”.

3. **List ≤3 bullet suggestions** for further improvement or data the author might add next time.

Output order for *each* narrative:
**Rating Table → Revised Narrative → Improvement Bullets**.  
Do not add chit-chat; be tersely helpful.

"""

UP_SHARPEN = """
Here are the draft mini-narratives from Stage 1:

{narrative}

Please apply the system instructions to sharpen each one.

The original excerpt from which these narratives were synthesized are shown below.

{excerpt}

"""

In [26]:
sys_msg = SP_SHARPEN
usr_msg = UP_SHARPEN.format(narrative=resp, excerpt=excerpt1)
refined = ask(sys_msg, usr_msg)
print(refined)


### Mini-Narrative 1: Rotation Dynamics

| Fidelity to Brief | Scientific Precision | Mechanistic Depth | Experimental Imagination | Narrative Punch |
|-------------------|----------------------|-------------------|--------------------------|-----------------|
| 8                 | 8                    | 7                 | 7                        | 6               |

#### Core Plot
In Drosophila hindgut morphogenesis, E-cadherin and Myo1D synergize to impart left-right (LR) asymmetry. E-cadherin mutations disrupt this, causing loss of cell boundary rotation and LR bias. Myo1D overexpression, however, heightens LR asymmetry, resulting in intensified cell boundary rotation. The interplay between these molecules orchestrates the precise rotations essential for normal development.

#### Mechanistic Highlights
- **E-cadherin**: Maintains junctional integrity, crucial for cell boundary rotational chirality.
- **Myo1D overexpression**: Enhances LR asymmetry via increased actomyosin contract

In [52]:
sINFERENCE = """
You are an expert developmental-biomechanics analyst.

INPUT: A single narrative about Drosophila hindgut rotation (Core Plot / Mechanistic Highlights / Open Questions).

OUTPUT:  
1. A Markdown table with **three columns** – *Component*, *Synthesised Mechanistic Inference*, *Testable Prediction*.  
   • Extract ≥6 key components (e.g., E-cadherin, Myo1D, chirality-induced contraction, cell sliding vs T1 remodeling, vertex-angle constraint, polarity cues, feedback loop).  
   • Each inference = 1-2 crisp sentences that explain *how* the component drives or modulates rotation dynamics.  
   • Each prediction = a concrete, falsifiable assay or perturbation.
   • Flag any biology you must infer with “(inferred)”.

2. **Integrative Picture** – one paragraph (≤120 words) weaving the components into a single causal circuit that explains robust LR morphogenesis.

Style rules:  
• Use precise biomechanical language (torque, shear, clutch, vortex, chirality index).  
• Avoid vague phrases; quantify or name an observable whenever possible.  
• No chit-chat or extra prose—just the table followed by the integrative paragraph.\
• Integrative Picture must be ≤ 120 words.
• Prohibit phrases such as ‘affect,’ ‘alter,’ ‘possibly,’ ‘potential.’ Use specific biomechanical verbs (shear, entrain, damp, propagate) and quantitative nouns (nN, rad·min⁻¹).
• For every Testable Prediction, specify (i) the experimental method and (ii) an expected numerical or categorical outcome (e.g., rotation velocity doubles, chirality index shifts from 0.1 → 0.4).
"""

uINFERENCE = """
Here is the draft narrative:

{narrative}

Generate the deeper mechanistic inference table and integrative picture as specified.

Here is the original text from which the narrative was drawn from:

{excerpt}
"""

In [31]:
print(resp)

**Core Plot**  
Within the intricate dance of Drosophila's hindgut morphogenesis, the choreography of cell boundary rotation plays a critical role. E-cadherin (E-cad) and Myosin 1D (Myo1D) preside over this molecular ballet, dictating the rotation direction and frequency of cell boundaries. These rotations, measured as counterclockwise (ccw) or clockwise (cw), influence hindgut dynamics. In the E-cad mutants, the usual left-right (LR) bias in rotations vanishes, whereas in Myo1D-overexpressing embryos, the bias amplifies. Through this rotational mechanism, cells are not merely positioned but are tasked with ensuring proper tissue configuration critical for developmental processes like hindgut rotation.

**Mechanistic Highlights**  
- **E-cadherin (E-cad)**: Crucial for maintaining cell adhesive integrity during boundary rotation.  
- **Myosin 1D (Myo1D)**: Drives augmented rotational bias, reinforcing directional tissue rotation.  
- **Cell Sliding Dynamics**: Influences boundary rotat

In [44]:
sMsg = sINFERENCE
uMsg = uINFERENCE.format(narrative=resp, excerpt=excerpt1)
nar = ask(sMsg, uMsg)
print(nar)

| Component                         | Synthesised Mechanistic Inference                                                                                                      | Testable Prediction                                                                                                                                                                                                                                                                                                                                                                                                                   |
|-----------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [54]:
# pip install python-docx first
from pathlib import Path
from docx import Document
from docx.shared import Pt
import re

# ----------------- CONFIG -----------------
OUT_PATH = Path("/Users/b/fantasiagenesis/crayon/output/hindgut_outputs.docx")
N_REPETITIONS = 3                         # passes per excerpt
CTRL_RE = re.compile(r'[\x00-\x08\x0b\x0c\x0e-\x1f]')   # XML-illegal chars
# ------------------------------------------

def clean(text: str) -> str:
    """Strip control characters that Word/XML cannot accept."""
    return CTRL_RE.sub("", text or "")

def add_heading(doc, text: str, level: int):
    run = doc.add_heading(level=level).add_run(clean(text))
    run.font.name = "Calibri"
    run.font.size = Pt(14 if level == 1 else 12)

def ensure_doc() -> Document:
    """Load existing .docx or start a fresh one."""
    return Document(OUT_PATH) if OUT_PATH.exists() else Document()

# install once (outside the script, e.g. in your venv or conda env):
#   pip install python-docx

from docx import Document
import os

FILE = "/Users/b/fantasiagenesis/crayon/output/hindgut_morphogenesis.docx"

def create_or_read(path: str) -> None:
    """
    Create a Word document at `path` if it doesn't exist;  
    otherwise open it and print its text.
    """
    if not os.path.isfile(path):
        # -------------- create ----------------
        doc = Document()
        doc.add_heading("Auto-generated report", level=1)
        doc.add_paragraph("Hello, world! This document was built with python-docx.")
        doc.save(path)
        print(f"Created new file: {path}")
    else:
        # -------------- read ------------------
        doc = Document(path)
        full_text = "\n".join(p.text for p in doc.paragraphs)
        print(f"Existing file detected: {path}\n--- contents ---\n{full_text}")




-	Rotation dynamics. 
-	Cell/tissue behaviors.
-	Perturbations to wildtype LR symmetry breaking.
-	Update rules of the mechanical network.
-   T1 transitions.
-   behaviors of the cell epithelial.
-   Geometric disorder, topological disorder to the cell array
-   What are the specifics? What are the specific observations? The specific theories? The words implied in the paper? The next words? 
-   Take a single paragraph. Make it a gem. 
-   What are single themes of papers around the hindgut? 
-   What can you change? 
-   The independent stories
-   The intellectual targets
-   Define cell angle pnematic? Shape pneumatic? tension pnuematic?
-   stress of the hindgut
-   mathematical equations that are useful in the parameterizing a surface. (the relationship between math and the hindgut)
-   treat these spaces as real. what would it take to tell the complete story of the hindgut using math? using logic? using images?
-   What are the assumptions that can be made at the short vs. long timescales? What are the implications for each? 
-   What is the definition of curvature?
-   “Given an evolution f shape over time, can we read off the stresses that caused shape change?”
-   What are the set of mathematical/physical questions that you can ask about the hindgut?
-   words: The active stress, the force balance, the latent stress, the viscous components, the elastic components? The frame of reference, the environment, Cell attractors? Energy landscape? Decision-making? 
The waddington landscape, high dimensional tensors of gene expression, low dimensional manifold, 
-   frame these questions in terms of cells, inference, and …
-   Quail gastrulation;  the evidence of the tensile ring has maximal contricle stress in the posterior, the profile of the gaussian does not change, the cells and tissue change. Is this the opposite? What is the relationship between this tissue? this event? 
-   How much information is involved in the hindgut?: The stiffness of the constrictions
-   What are the intuitions for the hindgut? What is the origin of this intuition?
-   What are the questions that arise from looking at calcium activity? What are the questions of calcium and constructions in the hindgut?
-   What are the questions between active stress and viscoelasticity?
-   implications of adding an soft/hard constraint?
-   What are the universal questions in the science? In biology? in the hindgut? which universals apply to the hindgut? 
-   What are the possible predictions of a simulation? What can you predict biologically that you can measure?
-   formalize differentiation in terms of probability?  framing of development under a growth maximizing regulatory strategy? 
-   step between waddington landscape formalization and optimal transport
-   What does optimal transport look like across the domains: economics, coal distribution, sand, biology?
-   Distance, dynamics, manifold, what is possible within genetics? 
-   The story around schrondinger’s bridge.
-   rewards of development? The costs? 
-   The morals, language, conversation of mathematical equations inside biology
-   what are the possible complete stories of hindgut development? 
-   what are teh possible complete tools of hindgut development? 
-   treat idea space as real space.

In [63]:
n = random.randint(0, len(hindgut)-1)
title, text = list(hindgut.keys())[n], list(hindgut.values())[n]
excerpt = sample_text(text, 10000)
print(title)

Myosin1D is an evolutionarily conserved regulator of animal left–right asymmetry


In [68]:

n = random.randint(0, len(hindgut)-1)
title, text = list(hindgut.keys())[n], list(hindgut.values())[n]
excerpt = sample_text(text, 10000)
print(title)
print("\n\n")


Chirality in Planar Cell Shape Contributes to Left-Right Asymmetric Epithelial Morphogenesis





In [74]:
for _ in range(5):
    n = random.randint(0, len(hindgut)-1)
    title, text = list(hindgut.keys())[n], list(hindgut.values())[n]
    excerpt = sample_text(text, 10000)
    act = ask("If I have a lightsheet imaging dataset of hindgut development over time, what is important to quantify/inquire in this dataset given the following information?", excerpt)
    print(act)

Here is a non-exhaustive checklist of the kinds of measurements and derived parameters you will want to extract from a lightsheet time-series of Drosophila hindgut in order to recapitulate the analyses in Nakajima et al. and to ask the same mechanistic questions.

1. Tissue-level metrics  
   a. Hindgut length (along the anterior–posterior or tubular axis) as a function of time → elongation rate  
   b. Hindgut rotation or twist angle (e.g. the angle of the curved hook relative to the embryo’s midline) → quantify LR bias  
   c. Dorsal–ventral (DV) depth of the tube (height) at 0 min versus later times → rule out ventral leaning  
   d. Curvature and torsion of the 3D tube centerline over time  

2. Single-cell segmentation and tracking  
   a. 3D segmentation of apical (or basal) cell surfaces in each time-point  
   b. Tracking of cell centroids, junctions (vertices) and cell neighbors over time  

3. Cell-level “sliding” (chiral cell boundary movement)  
   a. X-displacement of cell

In [73]:
for _ in range(5):
    n = random.randint(0, len(hindgut)-1)
    title, text = list(hindgut.keys())[n], list(hindgut.values())[n]
    excerpt = sample_text(text, 10000)
    act = ask("If I have a lightsheet imaging dataset of hindgut development over time, what are the hypothesis driven inquires to perform given the following information?", excerpt)
    print(act)

Here are a series of hypothesis-driven questions (and the image-analysis routines you’d use to address them) that you could tackle in your lightsheet time-lapse of hindgut morphogenesis.  In each case the idea is to define a clear “if … then …” prediction about vesicle behavior, cell shape, lumen expansion or flow, then extract the corresponding measurements from your 4D data and, where possible, compare wild-type versus perturbation.  

1.  Vacuole-mediated lumen expansion  
    Hypothesis: an apically directed population of large, fluid-filled vacuoles traffics to the hindgut lumen and delivers both membrane and fluid, driving lumen growth.  
    •  Segment and count vacuoles (by size thresholding) in each epithelial cell at early vs late stages.  
    •  Measure vacuole trajectories: compute speed, directionality (vector toward apical surface), and fusion events (coalescence of two tracks into one).  
    •  Prediction: vacuole number in posterior cells will drop as lumen expands; v

In [ ]:
# great shed light on this seed. the ones that grow, act